In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

pd.set_option("display.max_columns", None)
RANDOM_STATE = 42


In [4]:
train_df = pd.read_excel("./data/raw/train.xlsx")
test_df  = pd.read_excel("./data/raw/test.xlsx")

print(train_df.shape)
print(test_df.shape)


(16209, 21)
(5404, 20)


In [6]:
dup_counts = train_df["id"].value_counts()
print("Number of IDs with multiple rows:", (dup_counts > 1).sum())


Number of IDs with multiple rows: 99


In [15]:
# continuous numerical features -> mean
mean_features = [
    "bedrooms", "bathrooms",
    "sqft_living", "sqft_lot",
    "floors",
    "sqft_above", "sqft_basement",
    "sqft_living15", "sqft_lot15"
]

# ordinal / categorical-like -> mode
mode_features = [
    "grade", "condition", "view"
]

# binary -> max
max_features = [
    "waterfront", "yr_renovated" 
]


first_features = [
    "lat", "long", "yr_built"
]


In [16]:
agg_dict = {}

for col in mean_features:
    agg_dict[col] = "mean"

for col in mode_features:
    agg_dict[col] = lambda x: x.mode().iloc[0]

for col in max_features:
    agg_dict[col] = "max"

for col in first_features:
    agg_dict[col] = "first"

# target
agg_dict["price"] = "mean"


In [17]:
train_agg = (
    train_df
    .groupby("id", as_index=False)
    .agg(agg_dict)
)

print("After aggregation:", train_agg.shape)


After aggregation: (16110, 19)


In [18]:
test_agg = (
    test_df
    .groupby("id", as_index=False)
    .agg({k: v for k, v in agg_dict.items() if k != "price"})
)

print("Test after aggregation:", test_agg.shape)

Test after aggregation: (5396, 18)


In [19]:
train_agg["log_price"] = np.log1p(train_agg["price"])

In [21]:
CURRENT_YEAR = 2020

train_agg["house_age"] = CURRENT_YEAR - train_agg["yr_built"]
test_agg["house_age"]  = CURRENT_YEAR - test_agg["yr_built"]

train_agg["renovated_flag"] = (train_agg["yr_renovated"] > 0).astype(int)
test_agg["renovated_flag"]  = (test_agg["yr_renovated"] > 0).astype(int)


In [25]:
DROP_COLS = ["price"]
train_clean = train_agg.drop(columns=DROP_COLS)
test_clean  = test_agg.copy()


In [26]:
train_split, val_split = train_test_split(
    train_clean,
    test_size=0.2,
    random_state=RANDOM_STATE
)

print(train_split.shape, val_split.shape)


(12888, 21) (3222, 21)


In [28]:
FEATURES = [
    col for col in train_split.columns
    if col not in ["id", "log_price", "lat", "long"]
]

scaler = StandardScaler()

train_split[FEATURES] = scaler.fit_transform(train_split[FEATURES])
val_split[FEATURES]   = scaler.transform(val_split[FEATURES])
test_clean[FEATURES]  = scaler.transform(test_clean[FEATURES])

joblib.dump(scaler, "tabular_scaler.pkl")


['tabular_scaler.pkl']

In [29]:
assert train_split["id"].is_unique
assert val_split["id"].is_unique

print("Final train:", train_split.shape)
print("Final val:", val_split.shape)
print("Final test:", test_clean.shape)


Final train: (12888, 21)
Final val: (3222, 21)
Final test: (5396, 20)


In [34]:
train_split.to_csv("data/processed/train_processed.csv", index=False)
val_split.to_csv("data/processed/val_processed.csv", index=False)
test_clean.to_csv("data/processed/test_processed.csv", index=False)
